#### Task 2
Leia o arquivo pedidos.csv e agrupe todos os cliente que fizeram mais de 2 compras nos dias de black friday dos últimos três anos. Filtre todos os clientes que são menores de 30 anos e
coloque numa lista TODOS os códigos de pedido e a data em que foram efetuados. Adicione também a idade do cliente.
Dataset: ​ https://storage.googleapis.com/luizalabs-hiring-test/clientes_pedidos.csv

In [3]:
!mkdir -p dataset
!wget https://storage.googleapis.com/luizalabs-hiring-test/clientes_pedidos.csv
!mv clientes_pedidos.csv dataset/clientes_pedidos.csv
!ls dataset/*

--2020-11-11 23:23:44--  https://storage.googleapis.com/luizalabs-hiring-test/clientes_pedidos.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 2800:3f0:4004:808::2010, 2800:3f0:4004:809::2010, 2800:3f0:4004:801::2010, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|2800:3f0:4004:808::2010|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 28839038 (28M) [application/octet-stream]
Saving to: ‘clientes_pedidos.csv.2’

clientes_pedidos.cs 100%[===================>]  27.50M  5.57MB/s    in 5.1s    

2020-11-11 23:23:49 (5.35 MB/s) - ‘clientes_pedidos.csv.2’ saved [28839038/28839038]

dataset/clientes_pedidos.csv  dataset/wordcount.txt


### 1. Importando bibliotecas

In [1]:
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.sql.types import IntegerType
# from pyspark.sql.types import DataType, StructField, StructType
from pyspark.sql.functions import regexp_replace
from pyspark.sql.functions import col, lit, array, count
from pyspark.sql.functions import months_between, countDistinct
from pyspark.sql.functions import from_unixtime, current_date, date_format, concat, concat_ws, collect_list

### 2. Criando session

In [2]:
spark = SparkSession.builder \
    .master("local") \
    .appName("Task 2") \
    .getOrCreate()

### 3. Lendo arquivo .csv

In [3]:
df = spark.read.csv("dataset/clientes_pedidos.csv", header=True)

In [4]:
df.show()

+--------------------+--------------------+-----------------------+-----------+
|       codigo_pedido|      codigo_cliente|data_nascimento_cliente|data_pedido|
+--------------------+--------------------+-----------------------+-----------+
|bc8b03a005d5bf742...|b07af86a4a6870737...|    1985-12-04 00:00:00| 1542974527|
|19b0583adf75322cc...|eaaf6b26ef3b9712e...|    1979-11-14 00:00:00| 1542998573|
|58fafb698b6d343e0...|c69f2ab5fc61484d7...|    1989-07-25 00:00:00| 1543007822|
|79dd9f6c88ba32c97...|b4067845511997517...|    1953-12-14 00:00:00| 1542966096|
|968806d40adf6aa8c...|7eecbc06bfec32b80...|    1985-05-03 00:00:00| 1543000756|
|b8c6e74cf1b462489...|f240c43e82dfe3ca0...|    1980-04-16 00:00:00| 1542993637|
|5d91ea3b69a22d55b...|ae03fddbb707cb739...|    1991-11-18 00:00:00| 1542996459|
|5e48766af0ffdfb60...|156f291101e8ba36b...|    1974-01-04 00:00:00| 1543004877|
|f85c3fdcb417a3431...|4cf8efefa3c5aef67...|    1985-04-18 00:00:00| 1542997957|
|8ff2a83de50bb0695...|772b6ad0ae61400f6.

In [5]:
df.filter(col('codigo_cliente')== 'cd8020d629f546c47457701ff9bd1ab4').show()

+--------------------+--------------------+-----------------------+-----------+
|       codigo_pedido|      codigo_cliente|data_nascimento_cliente|data_pedido|
+--------------------+--------------------+-----------------------+-----------+
|100e4a76bd6e2b4fb...|cd8020d629f546c47...|    2011-11-11 00:00:00| 1542937716|
|bc3a955efe83c08ce...|cd8020d629f546c47...|    2011-11-11 00:00:00| 1543237405|
|4d7d64ca272e57cc7...|cd8020d629f546c47...|    2011-11-11 00:00:00| 1511368290|
|629a33a900bcc6d19...|cd8020d629f546c47...|    2011-11-11 00:00:00| 1511348448|
|f2f9a5a58b223f682...|cd8020d629f546c47...|    2011-11-11 00:00:00| 1511345762|
|6701780b374a46526...|cd8020d629f546c47...|    2011-11-11 00:00:00| 1511356541|
|bea6374e4c1b47589...|cd8020d629f546c47...|    2011-11-11 00:00:00| 1511370479|
|3f2a3bb0145e58a3c...|cd8020d629f546c47...|    2011-11-11 00:00:00| 1511363642|
|64409e7e66a784b49...|cd8020d629f546c47...|    2011-11-11 00:00:00| 1541793478|
|9b862a8363da3c181...|cd8020d629f546c47.

#### 3.1. Quantos clientes distintos?

In [6]:
df.select(countDistinct("codigo_cliente").alias('clientes_distintos')).show()

+------------------+
|clientes_distintos|
+------------------+
|            219535|
+------------------+



### 4. Criando colunas de idade do cliente e data do pedido

In [7]:
def createAgeDateColumns(df):
    return (df.withColumn("data_pedido_date", 
                          date_format(from_unixtime(col('data_pedido')), "yyyy-MM-dd"))
              .withColumn('idade',
                          (months_between(current_date(), col('data_nascimento_cliente'))/12).cast(IntegerType())))

df2 = createAgeDateColumns(df)
print(f"Quantidade de registros: {df2.count()}")
df2.show(5)

Quantidade de registros: 297309
+--------------------+--------------------+-----------------------+-----------+----------------+-----+
|       codigo_pedido|      codigo_cliente|data_nascimento_cliente|data_pedido|data_pedido_date|idade|
+--------------------+--------------------+-----------------------+-----------+----------------+-----+
|bc8b03a005d5bf742...|b07af86a4a6870737...|    1985-12-04 00:00:00| 1542974527|      2018-11-23|   34|
|19b0583adf75322cc...|eaaf6b26ef3b9712e...|    1979-11-14 00:00:00| 1542998573|      2018-11-23|   40|
|58fafb698b6d343e0...|c69f2ab5fc61484d7...|    1989-07-25 00:00:00| 1543007822|      2018-11-23|   31|
|79dd9f6c88ba32c97...|b4067845511997517...|    1953-12-14 00:00:00| 1542966096|      2018-11-23|   66|
|968806d40adf6aa8c...|7eecbc06bfec32b80...|    1985-05-03 00:00:00| 1543000756|      2018-11-23|   35|
+--------------------+--------------------+-----------------------+-----------+----------------+-----+
only showing top 5 rows



In [8]:
df2.filter(col('codigo_cliente')== '9ba43b1b796103b4f020481d169bfd25').count()

7106

### 5. Filtrando as compras realizadas nas últimas 3 Black Fridays
Dataframe com as colunas:
- codigo_pedido
- codigo_cliente
- data_nascimento_cliente
- data_pedido
- data_pedido_date
- idade

In [9]:
def filterShoppingBlackFriday(df):
    bf_date = ["2017-11-24",
               "2017-11-25",
               "2017-11-26",
               "2018-11-23",
               "2018-11-24",
               "2018-11-25",
               "2019-11-29",
               "2019-11-30",
               "2019-12-01"]
    return df.filter(col("data_pedido_date").isin(bf_date))

dfSalesJustBF = filterShoppingBlackFriday(df2)
print(f"Quantidade de registros: {dfSalesJustBF.count()}")
print(f"Data do pedido distintos: {dfSalesJustBF.select('data_pedido_date').distinct().collect()}")
dfSalesJustBF.show(2)

Quantidade de registros: 76771
Data do pedido distintos: [Row(data_pedido_date='2018-11-23'), Row(data_pedido_date='2017-11-25'), Row(data_pedido_date='2018-11-24'), Row(data_pedido_date='2017-11-24'), Row(data_pedido_date='2017-11-26'), Row(data_pedido_date='2018-11-25')]
+--------------------+--------------------+-----------------------+-----------+----------------+-----+
|       codigo_pedido|      codigo_cliente|data_nascimento_cliente|data_pedido|data_pedido_date|idade|
+--------------------+--------------------+-----------------------+-----------+----------------+-----+
|bc8b03a005d5bf742...|b07af86a4a6870737...|    1985-12-04 00:00:00| 1542974527|      2018-11-23|   34|
|19b0583adf75322cc...|eaaf6b26ef3b9712e...|    1979-11-14 00:00:00| 1542998573|      2018-11-23|   40|
+--------------------+--------------------+-----------------------+-----------+----------------+-----+
only showing top 2 rows



In [10]:
dfSalesJustBF.filter(col('codigo_cliente') == '52c69e3a57331081823331c4e69d3f2e').count()

4890

### 6. Criando colunas com a lista dos pedidos
A coluna lista dos pedidos é formada por um array de arrays. Com os valores de codigo do pedido e a data do pedido.

Dataframe com as colunas:
- codigo_cliente
- lista_pedidos

In [11]:
def createOrderList(df):
    dfStaging = df.withColumn("codigo_pedido_data", 
                               array(concat(col('codigo_pedido'), 
                                            lit(', '),
                                            col('data_pedido_date'))))
    dfStaging = dfStaging.drop('data_nascimento_cliente', 
                               'data_pedido', 
                               'codigo_pedido',
                               'idade')
    dfEnd = dfStaging.groupBy('codigo_cliente').agg(collect_list(col('codigo_pedido_data')).alias('lista_pedidos'))
    return dfEnd

dfOrderList = createOrderList(df2)
print(f"Quantidade de registros: {dfOrderList.count()}")
dfOrderList.show(5)
dfOrderList.first()

Quantidade de registros: 219535
+--------------------+--------------------+
|      codigo_cliente|       lista_pedidos|
+--------------------+--------------------+
|000ef444f2f0b80c2...|[[fa012b08aab3a1f...|
|0012c3e5bb4975132...|[[d6cc73d0e6c03d2...|
|004ddc230a904ef28...|[[1052a5ae66c12c8...|
|006a5a8c5bcb7ff8f...|[[368337a3b82d8e6...|
|00807ca387d9eaea1...|[[70f3a91ff9970c8...|
+--------------------+--------------------+
only showing top 5 rows



Row(codigo_cliente='000ef444f2f0b80c2988eb7408500f3a', lista_pedidos=[['fa012b08aab3a1f50b2695926e2dd35a, 2018-11-14']])

### 7. Filtrar clientes com mais de duas compras nos dias da Black Friday

Dataframe com as colunas:
- codigo_cliente
- numero_pedidos

In [12]:
def filterCustomerOrders(df):
    dfStaging = (df.groupBy(col('codigo_cliente')) 
                   .agg(count('data_pedido').alias('numero_pedidos'))
                   .orderBy("numero_pedidos", ascending=False))
    dfEnd = dfStaging.filter(dfStaging.numero_pedidos > 2)
    return dfEnd

dfClientTwoOrders = filterCustomerOrders(dfSalesJustBF)
dfClientTwoOrders = dfClientTwoOrders.drop('numero_pedidos')
print(f"Quantidade de registros: {dfClientTwoOrders.count()}")
print(f"Quantidade de registros distintos: {dfClientTwoOrders.select('codigo_cliente').distinct().count()}")
dfClientTwoOrders.show(5)

Quantidade de registros: 606
Quantidade de registros distintos: 606
+--------------------+
|      codigo_cliente|
+--------------------+
|52c69e3a573310818...|
|cede469d4e8b97c05...|
|9ba43b1b796103b4f...|
|741a1bc4411188ed7...|
|693f89ff05b3b8d34...|
+--------------------+
only showing top 5 rows



In [13]:
dfClientTwoOrders.first()

Row(codigo_cliente='52c69e3a57331081823331c4e69d3f2e')

### 8. Quantidade de todos os pedidos

Dataframe com as colunas:
- codigo_cliente
- numero_pedidos

In [14]:
def countOrders(df):
    dfEnd = (df.groupBy(col('codigo_cliente')) 
                   .agg(count('data_pedido').alias('numero_pedidos'))
                   .orderBy("numero_pedidos", ascending=False))
    return dfEnd

dfClientCountOrders = filterCustomerOrders(df2)
print(f"Quantidade de registros: {dfClientCountOrders.count()}")
print(f"Quantidade de registros distintos: {dfClientCountOrders.select('codigo_cliente').distinct().count()}")
dfClientCountOrders.show(5)

Quantidade de registros: 4741
Quantidade de registros distintos: 4741
+--------------------+--------------+
|      codigo_cliente|numero_pedidos|
+--------------------+--------------+
|52c69e3a573310818...|         11004|
|9ba43b1b796103b4f...|          7106|
|cede469d4e8b97c05...|          6699|
|741a1bc4411188ed7...|          2524|
|693f89ff05b3b8d34...|          2356|
+--------------------+--------------+
only showing top 5 rows



### 8. Filtrar clientes que são menores de 30 anos e compraram na Black Friday

In [15]:
def filterAgeUnderThirty(df):
    dfStaging = df.dropDuplicates(['codigo_cliente'])
    dfStaging =  dfStaging.filter(df.idade < 30)
    dfEnd = dfStaging.select("codigo_cliente", "idade")
    return dfEnd

dfUnderThirty = filterAgeUnderThirty(dfSalesJustBF)
print(f"Quantidade de registros: {dfUnderThirty.count()}")
print(f"Quantidade de clientes distintos: {dfUnderThirty.select('codigo_cliente').distinct().count()}")

dfUnderThirty.show(2)

Quantidade de registros: 9997
Quantidade de clientes distintos: 9997
+--------------------+-----+
|      codigo_cliente|idade|
+--------------------+-----+
|dea0b14cd8340da83...|   26|
|623f93cc06e99a3ab...|   24|
+--------------------+-----+
only showing top 2 rows



### 9. Juntando os dados

In [22]:
def joinDataframes(dfOrders, dfCount, dfOrdersList, dfAge):
    innerJoinAgeOrders = dfOrders.join(dfCount, 
                                    ["codigo_cliente"],
                                    "inner")
    innerJoinStaging = innerJoinAgeOrders.join(dfOrdersList, 
                                    ["codigo_cliente"],
                                    "inner")
    innerJoinEnd = innerJoinStaging.join(dfAge, 
                                           ["codigo_cliente"],
                                           "inner")
    return innerJoinEnd

In [23]:
dfResult = joinDataframes(dfClientTwoOrders, dfClientCountOrders, dfOrderList, dfUnderThirty)

In [24]:
dfResult.count()

117

In [26]:
dfResult = dfResult.orderBy("numero_pedidos", ascending=False)
dfResult.show()

+--------------------+--------------+--------------------+-----+
|      codigo_cliente|numero_pedidos|       lista_pedidos|idade|
+--------------------+--------------+--------------------+-----+
|9ba43b1b796103b4f...|          7106|[[63280517b0d1aa2...|   19|
|741a1bc4411188ed7...|          2524|[[422c2b8ae5b39c8...|   13|
|693f89ff05b3b8d34...|          2356|[[a6dfc8bdda0f99e...|   12|
|9436b0e2e66029690...|          1584|[[3558ea79905ec1a...|   11|
|a18363603cedb08e9...|           920|[[3e575ee5a1554fe...|   28|
|cd8020d629f546c47...|           421|[[4d7d64ca272e57c...|    9|
|f17149a5e4581fe1a...|           407|[[0c7892ca300bd21...|    9|
|909a27f471f41681b...|           221|[[60cf7dcb785c9d5...|   16|
|f5ae9dabf82378781...|           151|[[06e70d01f1087d4...|   15|
|18d6d2ac14593d9a6...|           112|[[15e668c90e8926c...|   27|
|6e507c74ff1053a4e...|            44|[[2188ca275b5e828...|   20|
|c2be12f7afe1673f3...|            39|[[d1205e9d5da8a22...|   20|
|337a53330d059a443...|   

In [27]:
dfResultToCSV = dfResult.withColumn("lista_pedidos", dfResult['lista_pedidos'].cast("string"))

In [28]:
dfResultToCSV.show()

+--------------------+--------------+--------------------+-----+
|      codigo_cliente|numero_pedidos|       lista_pedidos|idade|
+--------------------+--------------+--------------------+-----+
|9ba43b1b796103b4f...|          7106|[[63280517b0d1aa2...|   19|
|741a1bc4411188ed7...|          2524|[[422c2b8ae5b39c8...|   13|
|693f89ff05b3b8d34...|          2356|[[a6dfc8bdda0f99e...|   12|
|9436b0e2e66029690...|          1584|[[3558ea79905ec1a...|   11|
|a18363603cedb08e9...|           920|[[3e575ee5a1554fe...|   28|
|cd8020d629f546c47...|           421|[[4d7d64ca272e57c...|    9|
|f17149a5e4581fe1a...|           407|[[0c7892ca300bd21...|    9|
|909a27f471f41681b...|           221|[[60cf7dcb785c9d5...|   16|
|f5ae9dabf82378781...|           151|[[06e70d01f1087d4...|   15|
|18d6d2ac14593d9a6...|           112|[[15e668c90e8926c...|   27|
|6e507c74ff1053a4e...|            44|[[2188ca275b5e828...|   20|
|c2be12f7afe1673f3...|            39|[[d1205e9d5da8a22...|   20|
|337a53330d059a443...|   

In [29]:
dfResultToCSV.repartition(1).write.csv('result/task2', mode='overwrite', header=True)